In [65]:
!pip install pyvis==0.1.9
!pip install networkx==2.8 


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import numpy as np

In [67]:
df = pd.read_csv('EjecucionColab/relaciones_stanza_clarified.csv')

In [68]:
# Grafo de NetworkX
G = nx.DiGraph()

# Añadir nodos y aristas con etiquetas
for _, row in df.iterrows():
    G.add_node(row['Head'])
    G.add_node(row['Tail'])
    G.add_edge(row['Head'], row['Tail'], label=row['Relation'])
    
# Obtiene el grado (número de relaciones por nodo)
degree_dict = dict(G.degree())
    
# Notebook False si quiero verlo en web, True en el propio notebook
net = Network(height='750px', width='100%', notebook=False, directed=True)

# Calcula rangos para el heatmap
max_degree = max(degree_dict.values())
min_degree = min(degree_dict.values())

def color_heatmap(degree, min_d, max_d):
    # Escala logarítmica
    degree = np.log1p(degree)
    min_d = np.log1p(min_d)
    max_d = np.log1p(max_d)

    scale = (degree - min_d) / (max_d - min_d)
    r = 0
    # Para hacer progresion entre de 0080FF a 00FFFF a 00FF00
    if scale < 0.5:
        g = int(255 * scale + 128)
        b = 255
    else:
        g = 255
        b = int(255 * (1 - scale))
    # Progresion normal entre azul puro y verde puro:    
    # g = int(255 * scale)
    # b = int(255 * (1 - scale))
    return f'rgb({r},{g},{b})'


# Añadir nodos manualmente con color y tamaño personalizados
for node in G.nodes():
    degree = degree_dict[node]
    size = 20 + degree * 2 # Ajustar la escala de tamaño
    color = color_heatmap(degree, min_degree, max_degree)
    font_size = 20 + degree * 2  # Ajustar la escala de fuente
    net.add_node(node, size=size, color=color, shape='dot', title=f'{node}: {degree} relaciones', label=node, font={'size': font_size})


# Añadir aristas manualmente con etiquetas
for edge in G.edges(data=True):
    source_degree = degree_dict[edge[0]]
    width = 1 + source_degree * 0.1  # grosor proporcional
    color = color_heatmap(source_degree, min_degree, max_degree)
    font_size = 20 + source_degree * 0.2  # Ajustar la escala de fuente
    net.add_edge(edge[0], edge[1], label=edge[2]['label'], width=width, color=color, font={'size': font_size,})




# Mostrar botones de configuración si quieres permitir ajuste en runtime

# Ajustar físicas para mejor visualización
net.set_options("""
var options = {
  "nodes": {
    "borderWidth": 3,
    "borderWidthSelected": 5
  },
  "interaction": {
    "hover": true,
    "multiselect": true
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -30000,
      "centralGravity": 0.3,
      "springLength": 200,
      "springConstant": 0.02,
      "damping": 0.09
    },
    "minVelocity": 0.75
  },
  "layout": {
    "improvedLayout": true
  },
  "configure": {
    "enabled": true
  }
}
""")

#net.show_buttons(filter_=["physics", "layout", "edges", "nodes", "interaction"])

net.show("grafo.html")
